In [ ]:
# import pm4py
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
import pandas as pd
import itertools
from pm4py.objects.log.util import general as log_util
from pm4py.objects.log.util import xes
import matplotlib
import random
matplotlib.use('TkAgg')

from pm4py.algo.discovery.inductive import factory as inductive_miner
from pm4py.visualization.petrinet import factory as pn_vis_factory
from pm4py.algo.simulation.tree_generator.versions import ptandloggenerator as tree_gen_factory
from pm4py.objects.conversion.process_tree import factory as pt_conv_factory
from pm4py.algo.simulation.playout.versions import basic_playout
from pm4py.algo.filtering.log.variants import variants_filter


In [ ]:
parameters = {'mode':10,'min':6,'max':12}
tree = tree_gen_factory.apply(parameters=parameters)
gen_net, gen_im, gen_fm = pt_conv_factory.apply(tree)
graph_pn = pn_vis_factory.apply(gen_net, gen_im, gen_fm)
pn_vis_factory.view(graph_pn)
print(tree)

In [ ]:
# check if we can re-discover the generate petri net
gen_log = basic_playout.apply_playout(gen_net, gen_im, no_traces=1000, max_trace_length=100)

In [ ]:
# Mining for a Petri net
from pm4py.algo.discovery.inductive import factory as inductive_miner
# parameters={'noiseThreshold':0.4}
Re_net, Re_initial_marking, Re_final_marking = inductive_miner.apply(gen_log,parameters=parameters)
# Re_net, Re_initial_marking, Re_final_marking = inductive_miner.apply(gen_log)

# Petri net visualization
from pm4py.visualization.petrinet import factory as pn_vis_factory

gviz_pn = pn_vis_factory.apply(Re_net, Re_initial_marking, Re_final_marking)
pn_vis_factory.view(gviz_pn)

var=variants_filter.get_variants_sorted_by_count(variants_filter.get_variants(gen_log))
# len(var)
# print(var)

In [ ]:
from pm4py.evaluation.replay_fitness.versions import alignment_based
from pm4py.evaluation.precision.versions import etconformance_token
from pm4py.objects.petri.check_soundness import check_relaxed_soundness_net_in_fin_marking as soundness
# soundness(net, initial_marking, final_marking)
fitness_wholeNet = alignment_based.apply(gen_log, gen_net, gen_im, gen_fm)
precision_wholeNet = etconformance_token.apply(gen_log, gen_net, gen_im, gen_fm)
print(fitness_wholeNet)
print('precision:',precision_wholeNet)

In [ ]:
def Gen_ev(env, bcs, net, initial_marking, max_trace_length , concept_name ,Init_timestamp ):
    with bcs.request() as req:
        yield req

        trace = log_instance.Trace()
        trace.attributes["concept:name"] = str(concept_name)
        marking = copy(initial_marking)
        for j in range(100000):
            mu, sigma = 1, 0.4
#             mu, sigma = 4, 2
            g=np.random.normal(mu, sigma)
            dtime = g if g > 0 else mu
            dtime=1
            if not semantics.enabled_transitions(net, marking):
                break
            all_enabled_trans = semantics.enabled_transitions(net, marking)
            all_enabled_trans = list(all_enabled_trans)
            shuffle(all_enabled_trans)
            trans = all_enabled_trans[0]
            if trans.label is not None:
                event = log_instance.Event()
                event["case:concept:name"] = 'case_'+str(concept_name)
                event["concept:name"] = trans.label
                event["time:timestamp"] = Init_timestamp + timedelta(hours=env.now)
                trace.append(event)
                stream.append(event)
                yield env.timeout(dtime)
            marking = semantics.execute(trans, net, marking)
            if len(trace) > max_trace_length:
                break

In [ ]:
import simpy
from copy import copy
from random import shuffle
import pm4py.objects.log.log as log_instance
from pm4py.objects.petri import semantics
# time stamp interval = 1 sec
import pm4py.objects.log.log as log_instance
import time
import datetime
import pytz
from datetime import timedelta
Init_timestamp=datetime.datetime(2010, 1, 1, 0, 0, 0, 0, pytz.UTC)
stream = log_instance.EventStream()


max_trace_length=100
net=gen_net
initial_marking=gen_im
# How many cases to generate
no_traces=3000
# max parallel case
workers=150

env = simpy.Environment()
bcs = simpy.Resource(env, capacity=workers)
for i in range(no_traces):
    env.process(Gen_ev(env, bcs, net, initial_marking, 100 , i ,Init_timestamp ))
env.run()

In [ ]:
# plot figure of parallel cases vs. time step

from pm4py.objects.conversion.log.versions import to_event_log

log = to_event_log.transform_event_stream_to_event_log(stream,enable_deepcopy=True)
tr_len = {}
for case_index, case in enumerate(log):
    tr_len[case.attributes[xes.DEFAULT_TRACEID_KEY]]=log[case_index].__len__()
    
case_glue=log_util.CASE_ATTRIBUTE_GLUE
count_dic={}
num_act_list=[]
for event in stream:
    if event[case_glue] in count_dic:
        count_dic[event[case_glue]]=count_dic[event[case_glue]]-1
    else:
        count_dic[event[case_glue]]=tr_len[event[case_glue]]-1
    num_act=0
    for k,v in count_dic.items():
        if v !=0:
            num_act=num_act+1
    num_act_list.append(num_act)

plt.figure(figsize=(20,10))
plt.plot(num_act_list)
plt.ylabel('# active')
plt.show()

In [ ]:
from pm4py.objects.log.exporter.xes import factory as exporter
exporter.apply(stream,'test.xes')